# 1. k-NN: Classification

---
## Use k-NN

- k=3, L2 distance
- k=5, L2 distance

<br>

L2 distance means the Euclidean distance, and it is calculated as follows:

$$
\sqrt{\sum_{i=1}^{n} (x_i - y_i)^2}
$$
<br>
So we need to calculate the distance from (8, 9, 111) to all the given data points and then select the k nearest neighbors.

This is a classification problem, so we will use the majority vote to determine the label of the new data point.

The process is as follows:



In [5]:
import numpy as np

# Define the new data point
new_data_point = np.array([8, 9, 111])

# Define the given data for each class
class_A = np.array(
    [[7, 8, 200], [8, 9, 220], [6, 6, 180], [7.5, 8.5, 210], [6.5, 7, 190]]
)

class_B = np.array(
    [[5, 5, 100], [5.5, 4.5, 120], [6, 5, 130], [5, 6, 110], [5.5, 5, 115]]
)

class_C = np.array(
    [[2, 2, 50], [2.5, 2.5, 55], [1.5, 2, 45], [2, 2.5, 52], [3, 2, 58]]
)

class_D = np.array(
    [[10, 3, 120], [12, 3.1, 110], [11, 2.9, 130], [14, 3, 120], [15, 2.6, 110]]
)

# Function to calculate Euclidean distances
def calculate_distances(samples, new_sample):
    # Euclidean distance
    return np.sqrt(np.sum((samples - new_sample) ** 2, axis=1)) 


# Calculate distances for each class
distances_A = calculate_distances(class_A, new_data_point)
distances_B = calculate_distances(class_B, new_data_point)
distances_C = calculate_distances(class_C, new_data_point)
distances_D = calculate_distances(class_D, new_data_point)

# Print the distances
print("Distances for class A:", distances_A)
print("Distances for class B:", distances_B)
print("Distances for class C:", distances_C)
print("Distances for class D:", distances_D)
print("\n")

all_distances = np.concatenate((distances_A, distances_B, distances_C, distances_D)) 
# 5 data points for each class
class_labels = np.array(["A"] * 5 + ["B"] * 5 + ["C"] * 5 + ["D"] * 5)

# indices of the sorted distances
sorted_indices = np.argsort(all_distances)

# when k=3
# labels of the 3 nearest neighbors
k_3_labels = class_labels[sorted_indices[:3]]
# 1st array: unique labels, 2nd array: counts of each unique label
unique_arr_3, count_arr_3 = np.unique(k_3_labels, return_counts=True)
# the label with the maximum  (majority vote)
k_3_pred = unique_arr_3[np.argmax(count_arr_3)]

# when k=5
# labels of the 5 nearest neighbors
k_5_labels = class_labels[sorted_indices[:5]] 
# 1st array: unique labels, 2nd array: counts of each unique label
unique_arr_5, count_arr_5 = np.unique(k_5_labels, return_counts=True)
# the label with the maximum  (majority vote)
k_5_pred = unique_arr_5[np.argmax(count_arr_5)] 

print("(1) The labels from the nearest to the farthest:", class_labels[sorted_indices])
print("(2) And the distances from the new data point to the nearest to the farthest:"
      , all_distances[sorted_indices])
print("\n")
print("(a) The predicted label with k=3:", k_3_pred)
print("(b) The predicted label with k=5:", k_5_pred)

Distances for class A: [ 89.01123525 109.          69.09413868  99.00252522  79.03954706]
Distances for class B: [12.08304597 10.36822068 19.5192213   4.35889894  6.18465844]
Distances for class C: [61.69278726 56.64362277 66.68770501 59.65945022 53.69357503]
Distances for class D: [11.          7.19791637 20.17944499 12.36931688  9.53729521]


(1) The labels from the nearest to the farthest: ['B' 'B' 'D' 'D' 'B' 'D' 'B' 'D' 'B' 'D' 'C' 'C' 'C' 'C' 'C' 'A' 'A' 'A'
 'A' 'A']
(2) And the distances from the new data point to the nearest to the farthest: [  4.35889894   6.18465844   7.19791637   9.53729521  10.36822068
  11.          12.08304597  12.36931688  19.5192213   20.17944499
  53.69357503  56.64362277  59.65945022  61.69278726  66.68770501
  69.09413868  79.03954706  89.01123525  99.00252522 109.        ]


(a) The predicted label with k=3: B
(b) The predicted label with k=5: B


---
## Use weight-base k-NN. 

- k = 3 and weight is 𝑤 𝑥 = exp(−𝑑𝑖𝑠𝑡(𝑥, 𝑥!)) 
- k = 5 and weight is 𝑤 𝑥 = exp(−𝑑𝑖𝑠𝑡(𝑥, 𝑥!))

<br>

The weight-based k-NN classification is calculated as follows:

$$
\hat{y} = \arg\max_{y} \sum_{i=1}^{k} w_i \cdot I(y_i = y)
$$

where $w_i$ is the weight of the $i$-th nearest neighbor, and $I(y_i = y)$ is the indicator function that returns 1 if $y_i = y$ and 0 otherwise. 

(It just mean that choose the label of the nearest neighbor that has the highest weight sum.)

The process is as follows:

In [6]:
def calculate_weighted_label (labels, weights, k):
    # array of unique labels with k nearest neighbors
    unique_labels = np.unique(labels) 
    # array of zeros with the length of unique labels (to store the weighted scores of each label)
    weighted_votes = np.zeros(len(unique_labels))
    for i in range(k):
        label = labels[i]
        # index of the label in the unique_labels array
        index = np.where(unique_labels == label)
        # adding the weight of the label
        weighted_votes[index] += weights[i]
    print("The weighted score for %d nearest neighbors:"%k)
    for i in range(len(unique_labels)):
        print("%s: %f"%(unique_labels[i], weighted_votes[i]))
    # the label with the maximum weighted score (majority vote)
    return unique_labels[np.argmax(weighted_votes)] 

 # all_distances[sorted_indices] is the sorted distances
sorted_weights = np.exp(-all_distances[sorted_indices]) 
print("The weights of sorted data points (from the nearest to the farthest):"
      , sorted_weights)
print("\n")

weighted_k_3_pred = calculate_weighted_label(class_labels[sorted_indices[:3]]
                                             , sorted_weights, 3)
print("(c) The predicted label with k=3 and weight-base k-NN:"
      , weighted_k_3_pred)

print("\n")
weighted_k_5_pred = calculate_weighted_label(class_labels[sorted_indices[:5]]
                                             , sorted_weights, 5)
print("(d) The predicted label with k=5 and weight-base k-NN:", weighted_k_5_pred)

The weights of sorted data points (from the nearest to the farthest): [1.27924651e-02 2.06080532e-03 7.48143042e-04 7.21116306e-05
 3.14151381e-05 1.67017008e-05 5.65457292e-06 4.24691858e-06
 3.33357246e-09 1.72257601e-09 4.79928619e-24 2.51181239e-25
 1.23092042e-26 1.61124996e-27 1.09118328e-29 9.83550898e-31
 4.71585951e-35 2.20247864e-39 1.00867116e-43 4.59093847e-48]


The weighted score for 3 nearest neighbors:
B: 0.014853
D: 0.000748
(c) The predicted label with k=3 and weight-base k-NN: B


The weighted score for 5 nearest neighbors:
B: 0.014885
D: 0.000820
(d) The predicted label with k=5 and weight-base k-NN: B


---
## Let X=(a,b,c). Which factor in (a, b, c) would predominantly affect the class determination?

When features have different scales, distance calculations are heavily influenced by features with larger scales.

We use L2 distance when using k-NN, so the feature with the largest scale will predominantly affect the class determination.

(e) The factor `c` has the largest scale, so we can assume that `c` would predominantly affect the class determination.
